In [20]:
import pyproj
from pyproj import Transformer, CRS

In [15]:
transformer = Transformer.from_crs("EPSG:4326", "EPSG:4978")  # WGS84 → ECEF
X, Y, Z = transformer.transform(42.6977, 23.3219, 550)

In [18]:
X_pyproj, Y_pyproj, Z_pyproj = transformer.transform(42.6977, 23.3219, 550)

print(f"Координати X: {abs(X_pyproj):.3f} m, Y: {abs(Y_pyproj):.3f} m, Z: {abs(Z_pyproj):.3f} m")

Координати X: 4311575.420 m, Y: 1858811.720 m, Z: 4303253.781 m


In [22]:


# Създаване на CRS по различни начини
crs1 = CRS.from_epsg(4326)        # WGS84 geographic
crs2 = CRS.from_epsg(32635)       # UTM zone 35N
crs3 = CRS.from_string("EPSG:4326")
crs4 = CRS.from_proj4("+proj=longlat +datum=WGS84")
# crs5 = CRS.from_wkt("GEOGCS[...]")  # Well-Known Text - INVALID, so commented out

# Информация за CRS
print(f"Име: {crs1.name}")
print(f"Тип: {crs1.type_name}")
print(f"Единици: {crs1.axis_info}")
print(f"Област: {crs1.area_of_use}")
print(f"Датум: {crs1.datum}")

Име: WGS 84
Тип: Geographic 2D CRS
Единици: [Axis(name=Geodetic latitude, abbrev=Lat, direction=north, unit_auth_code=EPSG, unit_code=9122, unit_name=degree), Axis(name=Geodetic longitude, abbrev=Lon, direction=east, unit_auth_code=EPSG, unit_code=9122, unit_name=degree)]
Област: - name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Датум: World Geodetic System 1984 ensemble


In [23]:
# 3D трансформация (включва височина)
# WGS84 (географски) → ECEF (картезиански)
geo_to_ecef = Transformer.from_crs("EPSG:4326", "EPSG:4978", always_xy=True)

lon, lat, h = 23.3219, 42.6977, 550  # София, 550м н.в.
X, Y, Z = geo_to_ecef.transform(lon, lat, h)
print(f"ECEF: X={X:.1f}, Y={Y:.1f}, Z={Z:.1f}")

ECEF: X=4311575.4, Y=1858811.7, Z=4303253.8


In [24]:
from pyproj import Geod

# Създаване на геодезически обект
geod = Geod(ellps='WGS84')

# Разстояние и азимут между две точки
lon1, lat1 = 23.3219, 42.6977  # София
lon2, lat2 = 24.7536, 42.1354  # Пловдив

# Пряка задача - разстояние и азимути
azimuth12, azimuth21, distance = geod.inv(lon1, lat1, lon2, lat2)
print(f"Разстояние София-Пловдив: {distance/1000:.1f} км")
print(f"Азимут София→Пловдив: {azimuth12:.1f}°")
print(f"Азимут Пловдив→София: {azimuth21:.1f}°")

# Обратна задача - координати по азимут и разстояние
lon3, lat3, back_azimuth = geod.fwd(lon1, lat1, azimuth12, distance)
print(f"Изчислени координати: {lon3:.6f}, {lat3:.6f}")
print(f"Оригинални:           {lon2:.6f}, {lat2:.6f}")

Разстояние София-Пловдив: 133.4 км
Азимут София→Пловдив: 117.4°
Азимут Пловдив→София: -61.6°
Изчислени координати: 24.753600, 42.135400
Оригинални:           24.753600, 42.135400


In [25]:
# Площ на полигон (геодезическа)
# Координати на триъгълник София-Пловдив-Бургас
lons = [23.3219, 24.7536, 27.4626, 23.3219]  # затворен полигон
lats = [42.6977, 42.1354, 42.6960, 42.6977]

area, perimeter = geod.polygon_area_perimeter(lons, lats)
print(f"Площ: {abs(area)/1e6:.1f} кв.км")
print(f"Периметър: {perimeter/1000:.1f} км")

Площ: 10903.9 кв.км
Периметър: 704.1 км


In [26]:
def transform_large_dataset(lons, lats, from_crs, to_crs, chunk_size=10000):
    """Трансформира големи масиви данни на части"""
    transformer = Transformer.from_crs(from_crs, to_crs, always_xy=True)
    
    n_points = len(lons)
    xs = np.zeros(n_points)
    ys = np.zeros(n_points)
    
    for i in range(0, n_points, chunk_size):
        end = min(i + chunk_size, n_points)
        xs[i:end], ys[i:end] = transformer.transform(
            lons[i:end], lats[i:end]
        )
    
    return xs, ys

# Пример с 100,000 точки
n = 100000
random_lons = np.random.uniform(22, 29, n)  # България
random_lats = np.random.uniform(41, 45, n)

xs, ys = transform_large_dataset(random_lons, random_lats, "EPSG:4326", "EPSG:32635")
print(f"Трансформирани {len(xs)} точки")

Трансформирани 100000 точки
